# Loan Default Prediction

## Abstract

Loan default poses significant financial risk to lenders, motivating the development of predictive models that accurately flag high-risk borrowers. In this study, we leverage a publicly available LendingClub dataset of 396,000 loan records—each described by 27 borrower and loan characteristics (e.g., loan amount, annual income, credit utilization, purpose, bankruptcy history)—to predict whether a loan will be fully paid or charged off. After extensive data cleaning (deduplication, removal of weak predictors, imputation of missing values, and encoding of categorical features), we train and evaluate three classifiers: k-nearest neighbors (KNN), random forest (RF), and a two-layer multilayer perceptron (MLP) neural network. Models are compared using recall (to minimize costly false negatives), precision (to avoid rejecting creditworthy applicants), and the F1 score (to balance both error types). Our results show that the neural network achieves the highest F1 score (0.626) and precision (0.922), while KNN attains the highest recall (0.479). Random forest delivers stable performance (test accuracy 0.89) with moderate recall (0.45) and precision (0.88). We conclude that the neural network offers the best trade-off for default prediction, and we discuss future work on threshold tuning, hyperparameter optimization, and advanced imbalance-handling techniques.










## Introduction

### Problem Statement  
Loan default imposes significant financial risk on lenders by leading to direct monetary losses, higher funding costs, and operational burdens in collections. In this project, we aim to build a predictive model that, given a borrower’s application data, can accurately classify whether a loan will be **fully paid** or **charged off**.  

### Motivation & Significance  
- **Reducing Financial Losses.** By catching high‐risk loans before approval, lenders can minimize costly false negatives (i.e., missed defaults) and protect their balance sheets.  
- **Operational Efficiency.** Automated default prediction helps underwriters prioritize manual reviews and allocate capital more effectively.  
- **Broader Use Cases.**  
  - _Consumer Lending_: Banks and online platforms (e.g., LendingClub) can embed such models into their risk‐assessment pipelines.  
  - _Corporate & Government Borrowing_: Similar techniques can be adapted for small‐business loans or municipal bonds.  


## Setup

### Dataset Description  
- **Source:** LendingClub public dataset  
- **Records:** 396,030 loan applications  
- **Features (27 total):**  
  - **Numerical:** `loan_amnt`, `term`, `int_rate`, `installment`, `annual_inc`, `dti`, `earliest_cr_line`, `open_acc`, `pub_rec`, `revol_bal`, `revol_util`, `total_acc`, `mort_acc`, `pub_rec_bankruptcies`  
  - **Categorical (one-hot encoded):** `sub_grade`, `verification_status`, `purpose`, `initial_list_status`, `application_type`, `home_ownership`, `zip_code` dummies  
- **Target:** `loan_status_binary` (0 = non-default, 1 = default)  
- **Class Distribution:** 318,357 (81%) non-defaults, 77,673 (19%) defaults  

## Basic Statistics  
| Feature              |   Mean    |    Std    |
|----------------------|----------:|----------:|
| loan_amnt            | 14,113.89 |  8,357.44 |
| term                 |    41.70  |    10.21  |
| int_rate             |    13.64  |     4.47  |
| installment          |   431.85  |   250.73  |
| annual_inc           | 74,203.18 | 61,637.62 |
| dti                  |    17.38  |    18.02  |
| earliest_cr_line     | 1,997.86  |     7.20  |
| open_acc             |    11.31  |     5.14  |
| pub_rec              |     0.18  |     0.53  |
| revol_bal            | 15,844.54 | 20,591.84 |
| revol_util           |    53.79  |    24.44  |
| total_acc            |    25.41  |    11.89  |
| mort_acc             |     1.79  |     2.06  |
| pub_rec_bankruptcies |     0.12  |     0.36  |
| loan_status_binary   |     0.20  |     0.40  |

### Missing Values & Imputation  
| Feature                 | Missing Count | Missing % | Imputation Strategy                                                       |
|-------------------------|--------------:|----------:|----------------------------------------------------------------------------|
| **mort_acc**            |        37,795 |     9.54% | Fill with mean `mort_acc` grouped by `total_acc`                          |
| **emp_title**           |        22,927 |     5.79% | Drop (too many unique categories)                                          |
| **emp_length**          |        18,301 |     4.62% | Drop (low predictive power)                                                |
| **title**               |         1,756 |     0.44% | Drop (redundant with `purpose`)                                            |
| **pub_rec_bankruptcies**|           535 |     0.14% | Fill with mode (0)                                                         |
| **revol_util**          |           276 |     0.07% | Fill with overall median                                                  |

### Experimental Setup  
- **Train/Test Split:** 80/20 stratified by `loan_status_binary` (`random_state=666`)  
- **Cross-Validation:** 5-fold CV on training set for hyperparameter tuning  
- **Models & Hyperparameters:**  
  - **KNN:** k ∈ {1, 2, …, 10} (best k chosen by F1/recall)  
  - **Random Forest:**  
    - `n_estimators = 500`  
    - `max_features = 'sqrt'`  
    - `random_state = 666`  
  - **Neural Network (MLP):**  
    - **Architecture:** Input → 64 ReLU → 32 ReLU → Output  
    - **Loss:** Binary cross-entropy  
    - **Optimizer:** Adam, lr = 1e-3  
    - **Random seed:** 666  


## Results

Our three classifiers exhibit distinct strengths:

| Model                    | Accuracy | Recall | F1 Score |
|--------------------------|---------:|----------:|-------:|
| **KNN (k = 9)**          | 0.8323    | 0.4789 | 0.6080 |   
| **Random Forest (500 trees)** | 0.8800    | 0.4500 | 0.6100   |
| **Neural Network (64→32)**    | 0.9220   | 0.4750 | 0.6255   |

- The **Neural Network** achieves the highest F1 (0.6255) and precision (0.9222), indicating strong overall discrimination and few false positives.  
- **KNN** attains the highest recall (0.4789), maximizing detection of true defaulters at the expense of more false positives.  
- **Random Forest** delivers the second highest accuracy (0.88) and a solid F1 (0.61), reflecting robust, stable performance.


## Discussion

Our three classifiers strike different balances between precision and recall when predicting loan defaults on the LendingClub dataset. The **neural network** achieved the highest F1 score (0.6255) and precision (0.9222), indicating strong discrimination and few false positives. However, its recall (0.48) remains well below the weighted recall of 0.89 reported by the Stanford CS229 LendingClub project (Stanford University, 2018). That work also leveraged balanced sampling (e.g., SMOTE) and model ensembling, suggesting that similar targeted oversampling or stacked ensembles could substantially boost default detection.

The **random forest** (F1 = 0.610, precision ≈ 0.88) is consistent with findings by Li and Zhang (2019), who demonstrated that random forests outperform logistic regression and single decision trees in P2P loan default prediction. Yet its recall (0.45) could be improved by explicitly handling class imbalance through cost-sensitive losses or by tuning the classification threshold away from 0.5.

The **KNN** model yields the highest recall (0.4789) among our methods, but at the cost of lower precision (0.8323). Yanxiali (2021) compared KNN to more complex learners on LendingClub data and found that applying distance-weighting and feature selection can further improve recall.

Overall, although our models achieve competitive F1 scores, recall remains modest. We recommend that future work emphasize:  
1. **Class‐imbalance techniques** (SMOTE, ADASYN, class‐weighted losses)  
2. **Threshold tuning** to optimize the trade-off between recall and precision  
3. **Ensemble methods** (e.g., stacking Random Forest, Neural Network, and XGBoost) to leverage complementary strengths  


Overall, while our models achieve competitive F1 scores, recall remains modest. We recommend future efforts focus on:  
1. **Class‐imbalance techniques** (SMOTE, ADASYN, class‐weighted losses)  
2. **Threshold tuning** to optimize recall vs. precision trade-offs  
3. **Model ensembles** (stacking RF, NN, XGBoost) to leverage complementary strengths  

# Conclusion

We processed 396 K LendingClub loan records—cleaning, imputing, and encoding 27 features—and trained three classifiers (KNN, random forest, two-layer neural network) using stratified splits and 5-fold CV. By comparing accuracy, precision, recall, and F1, we found the neural network offers the best precision and F1 (0.9222, 0.6255), KNN the best recall (0.4789), and random forest the most stable accuracy (0.88) with solid F1 (0.61). These results demonstrate the value of non-linear models for default prediction, and point to class-imbalance handling and ensemble strategies as the next steps for improving defaulter detection.  


## References

De Dieu Nyandwi, J. D. (2020). *Lending Club dataset* [Data set]. Kaggle. https://www.kaggle.com/datasets/jeandedieunyandwi/lending-club-dataset

Li, X., & Zhang, Y. (2019). A study on predicting loan default based on the random forest. _Procedia Computer Science_, _162_, 130–137. https://doi.org/10.1016/j.procs.2019.12.017

Stanford University. (2018). *LendingClub Loan Default and Profitability Prediction* [CS229 project]. Retrieved from https://cs229.stanford.edu/proj2018/poster/69.pdf

yanxiali. (2021). *Predicting Default Clients of Lending Club Loans* [Source code]. GitHub. https://github.com/yanxiali/predicting-default-clients-of-lending-club-loans
